In [1]:
import numpy as np
from sys import maxsize
import random

*R* : Set of *m* resources represented as (id, cost?) tuples

In [2]:
m = 10

R = [(i, np.random.randint(0, 100)) for i in range(m)]
print(R)

[(0, 68), (1, 55), (2, 61), (3, 5), (4, 53), (5, 49), (6, 35), (7, 53), (8, 34), (9, 21)]


*N* : Set of *n* agents as (id, balance?, priority?)

In [3]:
n = 2

N = [(i, np.random.randint(0, 100), np.random.randint(0, 100)) for i in range(n)]
N

[(0, 76, 80), (1, 80, 52)]

Zero Allocations

In [4]:
def init_allocs(resources, agents):
    allocs = []
    for index, i in enumerate(agents):
        r_i = [0 for x in resources]
        allocs.append(r_i)
        
    return allocs

A = init_allocs(R, N)
print(A)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


Random allocations

In [5]:
def rand_allocs(resources, agents):
    allocs = init_allocs(resources, agents)
    for index, i in enumerate(resources):
        allocs[np.random.randint(0, 100)//(100//len(agents))][index] = 1
    return allocs

In [6]:
A = rand_allocs(R, N)
print(A)

[[1, 1, 0, 0, 1, 1, 0, 0, 1, 0], [0, 0, 1, 1, 0, 0, 1, 1, 0, 1]]


Random utility values of resources for both users : U

In [7]:
U = []

for index, i in enumerate(N):
    u_i = random.sample(range(0, 100), len(R))
    U.append(u_i)
    
    
print(U)

[[48, 74, 64, 20, 79, 2, 28, 54, 4, 18], [37, 14, 75, 90, 16, 26, 98, 88, 50, 49]]


Random pair-wise complementing utility values : V

In [8]:
V = []

for index, i in enumerate(N):
    v_ijjp1 = random.sample(range(0, 100), len(R)//2)
    V.append(v_ijjp1)
    
    
print(V)

[[27, 69, 48, 76, 59], [98, 26, 3, 31, 43]]


In [9]:
additive_utililty = lambda alloc, utility : sum(np.array(alloc) * np.array(utility))

def complementing_utility(alloc, utility, p_utility) :
    pairwise_alloc = []
    for i in range(0, len(alloc), 2):
        pairwise_alloc.append(alloc[i] & alloc[i+1])
        
#     print(alloc)
#     print(pairwise_alloc)
#     print(utility)
#     print(p_utility)
    return additive_utililty(alloc, utility) + additive_utililty(pairwise_alloc, p_utility)

In [36]:
print(additive_utililty(A[0], U[0]))
print(complementing_utility(A[0], U[0], V[0]))

207
282


In [34]:
V[0]

[27, 69, 48, 76, 59]

In [11]:
def optimal_nash_prod_wcomp(utility, p_utility) :
    n = len(utility)
    if n :
        m = len(utility[0])
        
    optimal_alloc = np.zeros((n, m))
    opt_payoff = 0
    for i in range(2**(m*n)) :
        curr_alloc = np.array([int(x) for x in list(np.binary_repr(i, width=(m*n)))]).reshape(n, m).tolist()
        req_resources = np.zeros(m)
        payoff = 1
        # print(curr_alloc)
        for j in range(len(curr_alloc)) :
            req_resources += curr_alloc[j]
        if not np.array_equal(req_resources, np.ones(m)) :
            continue
        for j in range(len(curr_alloc)) :
            payoff *= complementing_utility(curr_alloc[j], utility[j], p_utility[j])
            
        if opt_payoff < payoff :
            opt_payoff = payoff
            optimal_alloc = curr_alloc
            
    return(optimal_alloc)

In [12]:
onpa = optimal_nash_prod_wcomp(U, V)

<hr>
#### Envy and envy up to one good calculations
<hr>

In [52]:
def envyMap(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)):
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[j], p_utilities[j])
    return envy_map

In [53]:
def envyMap_upto1(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    size_res = np.array(allocs).shape[1]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)) :
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[i], p_utilities[i])
            if envy_map[i][j] < 0 :
                envy_res_drop1_j = np.zeros(size_res)
                max_pos_envy = - maxsize
                for x in range(len(envy_res_drop1_j)) :
                    new_alloc_j = list(allocs[j])
                    new_alloc_j[x] = 0
                    envy_res_drop1_j[x] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
                    - complementing_utility(new_alloc_j, utilities[i], p_utilities[i])
                    # print(envy_res_drop1_j)
                    if envy_res_drop1_j[x] >= max_pos_envy:
                        envy_map[i][j] = envy_res_drop1_j[x]
                        max_pos_envy = envy_res_drop1_j[x]
                
    return envy_map

In [51]:
print(U)
print(V)

[[48, 74, 64, 20, 79, 2, 28, 54, 4, 18], [37, 14, 75, 90, 16, 26, 98, 88, 50, 49]]
[[27, 69, 48, 76, 59], [98, 26, 3, 31, 43]]


Envy map represents the envy between two agents. $envymap_{ij}$ is the amount with which i envies j

In [62]:
rand_a = rand_allocs(R, N)
print("Random allocations : ", rand_a)
print(envyMap(rand_a, U, V))
print(envyMap_upto1(rand_a, U, V))

Random allocations :  [[0, 0, 1, 1, 1, 1, 0, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 0, 1, 1]]
[[  0. -53.]
 [ 53.   0.]]
[[ 0. 78.]
 [65.  0.]]


In [63]:
print("Nash product allocations : ", onpa)
print(envyMap(onpa, U, V))
print(envyMap_upto1(onpa, U, V))

Nash product allocations :  [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1]]
[[  0.  72.]
 [-72.   0.]]
[[  0. 192.]
 [109.   0.]]


<hr>
#### Envy free experiments on large sample
<hr>

m => Number of resources <br>
R => Resources <id, value> <br>
n => Number of agents <br>
N => Agents <id, value, priority> <br>
U => Additive utilities
V => Complementing Utilities

Additive utilities

In [112]:
np.random.seed(10)

ef1_tracker = []

for x in range(1000):
    m = 10
    R = [(i, np.random.randint(0, 100)) for i in range(m)]

    n = 2
    N = [(i, np.random.randint(0, 100), np.random.randint(0, 100)) for i in range(n)]

    U = []
    for index, i in enumerate(N):
        u_i = np.random.random_sample(len(R)) * 100
        U.append(u_i)

    V = []
    for index, i in enumerate(N):
        v_ijjp1 = np.random.random_sample(len(R)//2) * 0
        V.append(v_ijjp1)

    random_alloc = rand_allocs(R, N)
    envymap1 = envyMap_upto1(random_alloc, U, V)
    ef1_tracker.append(not bool(np.sum(envymap1<0)))
    
    # print("Allocation : ", random_alloc)
    # print(envymap1)
    # print("Is EF1 allocation? ", ef1_tracker[x])
    
print("EF1 percent : ", np.sum(ef1_tracker)/len(ef1_tracker) * 100)

EF1 percent :  36.3


Complementing utilities

In [113]:
np.random.seed(10)
for x in range(1000):
    m = 10
    R = [(i, np.random.randint(0, 100)) for i in range(m)]

    n = 2
    N = [(i, np.random.randint(0, 100), np.random.randint(0, 100)) for i in range(n)]

    U = []
    for index, i in enumerate(N):
        u_i = np.random.random_sample(len(R)) * 100
        U.append(u_i)

    V = []
    for index, i in enumerate(N):
        v_ijjp1 = np.random.random_sample(len(R)//2) * 100
        V.append(v_ijjp1)

    random_alloc = rand_allocs(R, N)
    envymap1 = envyMap_upto1(random_alloc, U, V)
    ef1_tracker.append(not bool(np.sum(envymap1<0)))
    
    # print("Allocation : ", random_alloc)
    # print(envymap1)
    # print("Is EF1 allocation? ", ef1_tracker[x])
    
print("EF1 percent : ", np.sum(ef1_tracker)/len(ef1_tracker) * 100)

EF1 percent :  37.9
